In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

data_path = Path("data")

### Utility Funcs

In [2]:
def open_diversity_sets(data_path, file_prefix, observations = "sequence", label = "consensus_stability_score"):
    sets = dict()
    
    for csv_f in data_path.glob(f"stability_diversity_train_{file_prefix}*"):
        parts = csv_f.stem.split("_")
        size = parts[-1]
        
        a_set = pd.read_csv(csv_f)
        
        to_drop_idx = []
        to_rename_label = None
        to_rename_obs = None
        for i, col in enumerate(a_set.columns):
            if "Unnamed" in col:
                to_drop_idx.append(i)
                
            elif label == col:
                to_rename_label = i
                
            elif observations == col:
                to_rename_obs = i
        
        a_set = a_set.drop(a_set.columns[to_drop_idx], axis=1)
        a_set = a_set.rename(columns={label: "label"})
        
        new_cols = ["label", observations, "diversity"]
        
        sets["train"] = a_set[new_cols]
        
    return sets

## What's the minimum amount of data to achieve r2 > 0.7

In [3]:
diversity_sets = open_diversity_sets(data_path, "1000")
diversity_sets["train"]

,label,sequence,diversity
0,-0.03,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK,0.879509
1,1.15,GSSGSLSDEDFKAVFGMTRSAFAMLPLWKQQNLKKEKGLFGSS,0.879126
2,0.74,TELKKKLEEALKKGEEVRVKFNGIEIRITSEDTARKAVELLEK,0.879500
3,0.73,GMADEEKLPPGWEKRMSRSSGRVYYTNHITNASQWERPSGGSS,0.879761
4,1.35,GMADEEKLPPGWEKRMSYSSGRVYYFNHITNASQWERPSGGSS,0.879780
...,...,...,...
996,0.84,GSSGSLSDNDFKAVFGMTRSAFANLPLWKQQNLKKEKGLFGSS,0.881958
997,0.80,TELKKKLEEALKKGEEVRVKFNGIEIRIESEDAARKAVELLEK,0.879496
998,0.86,GSSGSLSDESFKAVFGMTRSAFANLPLWKQQNLKKEKGLFGSS,0.880492
999,0.95,TELKKKLEEALKKGEEVRVKFNGIEIRITSEDAWRKAVELLEK,0.879480


### Load Protein Embeddings

In [4]:
from utils import load_dataset
X, y, dset = load_dataset(data_path, to_torch=True)

Using cache found in /home/step/.cache/torch/hub/facebookresearch_esm_master


In [5]:
is_in_diversity =  lambda seq_bytes: seq_bytes.decode("utf-8") in diversity_sets["train"]["sequence"].values

idxes = [idx for idx, seq in enumerate(dset["sequences"]) if is_in_diversity(seq)]
dset.close()
X, y = X[idxes], y[idxes]

### Load model

In [6]:
from skorch.callbacks import LoadInitState, Checkpoint, LRScheduler, EarlyStopping

from skorch import NeuralNetRegressor

from models import ProteinMLP

cp = Checkpoint(dirname='models')
load_state = LoadInitState(cp)

net = NeuralNetRegressor(ProteinMLP)

net.initialize()
net.load_params(checkpoint=cp)

y_pred = net.predict(X)
y[:10]

tensor([[-2.7473],
        [ 0.2262],
        [-0.8070],
        [-0.8322],
        [ 0.7302],
        [-2.8481],
        [-2.0417],
        [-0.8322],
        [-1.0086],
        [-0.8574]])

## Experiments

In [26]:
def setup(data_path, s, epochs=int(1e4), ckpt_dir = "", load_model = False):
    diversity_sets = open_diversity_sets(data_path, s)

    is_in_diversity =  lambda seq_bytes: seq_bytes.decode("utf-8") in diversity_sets["train"]["sequence"].values
    
    X, y, dset = load_dataset(data_path, to_torch=True)
    idxes = [idx for idx, seq in enumerate(dset["sequences"]) if is_in_diversity(seq)]
    X, y = X[idxes], y[idxes]
    dset.close()

    cb = Checkpoint(dirname=f"models/{ckpt_dir}")
    sched = LRScheduler(step_every="batch")
    stopper = EarlyStopping(patience=15)
    
    net = NeuralNetRegressor(
        ProteinMLP,
        max_epochs=epochs,
        lr=3e-3,
        iterator_train__shuffle=True,
        device='cuda',
        callbacks=[cb, sched, stopper],
    )
    
    if load_model:
        load_state = LoadInitState(cp)
        
        net.initialize()
        net.load_params(checkpoint=cp)
    
    return {"model": net, "X": X, "y": y}

In [20]:
exp = setup(data_path, "1000", epochs=1000, ckpt_dir=f"full")
        
X_train, y_train, dset = load_dataset(data_path, kind = 'train', reduce = False, to_torch = True)
dset.close()

exp["model"].fit(X_train, y_train)

Using cache found in /home/step/.cache/torch/hub/facebookresearch_esm_master
Using cache found in /home/step/.cache/torch/hub/facebookresearch_esm_master


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.7158        0.5840     +  0.0763
      2        0.6507        0.6294        0.0754
      3        0.6342        0.5737     +  0.0741
      4        0.6217        0.5768        0.0743
      5        0.6125        0.5583     +  0.0749
      6        0.6083        0.5519     +  0.0746
      7        0.6063        0.5433     +  0.0740
      8        0.5974        0.5988        0.0743
      9        0.5902        0.5290     +  0.0742
     10        0.5668        0.5119     +  0.0740
     11        0.5584        0.4996     +  0.0741
     12        0.5501        0.5010        0.0745
     13        0.5500        0.4981     +  0.0744
     14        0.5416        0.4691     +  0.0749
     15        0.5103        0.4311     +  0.0749
     16        0.4750        0.4072     +  0.0748
     17        0.4414        0.3735     +  0.0748
     18        0.4054        0.3520     +  0.0749


    139        0.1973        0.1848     +  0.0746
    140        0.1924        0.1885        0.0750
    141        0.1936        0.1846     +  0.0747
    142        0.1922        0.1853        0.0769
    143        0.1917        0.1924        0.0741
    144        0.1903        0.1835     +  0.0746
    145        0.1892        0.1821     +  0.0758
    146        0.1921        0.1879        0.0768
    147        0.1921        0.1783     +  0.0755
    148        0.1911        0.1810        0.0762
    149        0.1885        0.1907        0.0756
    150        0.1881        0.1784        0.0747
    151        0.1833        0.1848        0.0745
    152        0.1871        0.1785        0.0745
    153        0.1843        0.1778     +  0.0741
    154        0.1824        0.1836        0.0761
    155        0.1860        0.1879        0.0747
    156        0.1844        0.1796        0.0745
    157        0.1831        0.1871        0.0742
    158        0.1813        0.1836        0.0754


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=ProteinMLP(
    (fc1): Linear(in_features=1280, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=512, bias=True)
    (fc3): Linear(in_features=512, out_features=1, bias=True)
    (drop): Dropout(p=0.7, inplace=False)
    (act): LeakyReLU(negative_slope=0.01)
  ),
)

In [21]:
X_test, y_test, dset = load_dataset(data_path, kind = 'test', reduce = False, to_torch = True)
dset.close()

exp["model"].score(X_test, y_test)

Using cache found in /home/step/.cache/torch/hub/facebookresearch_esm_master


0.8260408952518665

In [27]:
from tqdm.autonotebook import tqdm
import pickle

def run_experiment(data_path, epochs = int(1e4), sizes = ["450", "700", "1000", "1500", "2000", "3000", "4000", "5000", "6000"]):
    for s in tqdm(sizes):
        exp = setup(data_path, s, epochs=epochs, ckpt_dir=f"diversity_{s}")
                
        exp["model"].fit(exp["X"], exp["y"])
        X_test, y_test, dset = load_dataset(data_path, kind = 'test', reduce = False, to_torch = True)
        dset.close()
        
        score = net.score(X_test, y_test)
        
        print(f"TEST R2: {round(score, 3)}")
        
        pickle.dump(exp["model"].history, open(Path("logs") / f"ProteinMLP_diversity_{s}_history.pkl", "wb"))
        pickle.dump(score, open(Path("logs") / f"ProteinMLP_diversity_{s}_test.pkl", "wb"))

In [ ]:
run_experiment(data_path=data_path)

  0%|          | 0/9 [00:00<?, ?it/s]

Using cache found in /home/step/.cache/torch/hub/facebookresearch_esm_master


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.7029        0.6411     +  0.0060
      2        0.6405        0.6407     +  0.0055
      3        0.6406        0.6406     +  0.0054
      4        0.6402        0.6405     +  0.0058
      5        0.6397        0.6399     +  0.0049
      6        0.6392        0.6393     +  0.0049
      7        0.6386        0.6389     +  0.0048
      8        0.6379        0.6386     +  0.0049
      9        0.6388        0.6385     +  0.0051
     10        0.6377        0.6384     +  0.0049
     11        0.6374        0.6383     +  0.0049
     12        0.6377        0.6376     +  0.0049
     13        0.6365        0.6368     +  0.0049
     14        0.6357        0.6361     +  0.0049
     15        0.6350        0.6354     +  0.0056
     16        0.6344        0.6348     +  0.0048
     17        0.6346        0.6343     +  0.0049
     18        0.6339        0.6338     +  0.0047


    126        0.4297        0.4026     +  0.0058
    127        0.4411        0.3987     +  0.0048
    128        0.4315        0.3945     +  0.0050
    129        0.4277        0.3911     +  0.0049
    130        0.4227        0.3876     +  0.0055
    131        0.4263        0.3841     +  0.0049
    132        0.4202        0.3809     +  0.0049
    133        0.4195        0.3774     +  0.0048
    134        0.4120        0.3741     +  0.0053
    135        0.4153        0.3713     +  0.0048
    136        0.4147        0.3683     +  0.0050
    137        0.4149        0.3656     +  0.0049
    138        0.4071        0.3630     +  0.0050
    139        0.3989        0.3604     +  0.0052
    140        0.4109        0.3583     +  0.0049
    141        0.4006        0.3561     +  0.0048
    142        0.4050        0.3542     +  0.0049
    143        0.3973        0.3523     +  0.0050
    144        0.3994        0.3503     +  0.0049
    145        0.3998        0.3486     +  0.0049


    263        0.3728        0.3047     +  0.0058
    264        0.3746        0.3044     +  0.0051
    265        0.3659        0.3043     +  0.0051
    266        0.3626        0.3042     +  0.0059
    267        0.3827        0.3044        0.0050
    268        0.3765        0.3040     +  0.0048
    269        0.3624        0.3036     +  0.0057
    270        0.3761        0.3034     +  0.0051
    271        0.3670        0.3032     +  0.0053
    272        0.3723        0.3030     +  0.0063
    273        0.3646        0.3027     +  0.0058
    274        0.3726        0.3024     +  0.0052
    275        0.3647        0.3020     +  0.0055
    276        0.3718        0.3018     +  0.0058
    277        0.3666        0.3014     +  0.0050
    278        0.3652        0.3014     +  0.0049
    279        0.3712        0.3014     +  0.0049
    280        0.3662        0.3013     +  0.0057
    281        0.3772        0.3012     +  0.0051
    282        0.3660        0.3012     +  0.0050


...

Using cache found in /home/step/.cache/torch/hub/facebookresearch_esm_master


TEST R2: 0.001


Using cache found in /home/step/.cache/torch/hub/facebookresearch_esm_master


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.8658        0.7321     +  0.0301
      2        0.7699        0.6942     +  0.0312
      3        0.7811        0.6884     +  0.0290
      4        0.7687        0.6878     +  0.0290
      5        0.7712        0.6836     +  0.0291
      6        0.7649        0.6809     +  0.0300
      7        0.7594        0.6792     +  0.0290
      8        0.7575        0.6783     +  0.0297
      9        0.7563        0.6786        0.0292
     10        0.7517        0.6694     +  0.0298
     11        0.7450        0.6634     +  0.0293
     12        0.7409        0.6638        0.0304
     13        0.7326        0.6564     +  0.0288
     14        0.7301        0.6531     +  0.0303
     15        0.7301        0.6520     +  0.0293
     16        0.7296        0.6515     +  0.0300
     17        0.7243        0.6515     +  0.0298
     18        0.7206        0.6610        0.0294


Using cache found in /home/step/.cache/torch/hub/facebookresearch_esm_master


In [25]:
pickle.load(open("logs/ProteinMLP_diversity_6000_test.pkl", "rb"))

0.0010775796472866084